In [7]:
import pandas as pd
import pyproj

ImportError: No module named pyproj

In [4]:
data = pd.read_csv('parking_regulation.csv')
data.head()

,geometry_simplified,geometry_simplified_vertex_count,geometry,geometry_vertex_count,OBJECTID,SG_KEY_BOR,SG_ORDER_N,SG_SEQNO_N,SG_MUTCD_C,SR_DIST,SG_SIGN_FC,SG_ARROW_D,x,y,SIGNDESC1,SG_KEY_BOR_ft_style,SG_SIGN_FC_ft_style,SG_ARROW_D_ft_style,import_notes
0,NaN,NaN,"<Point><coordinates>-73.9223354763,40.83625571...",1,11494927,B,P-132428,3,SP-287B,45,NaN,NaN,1005740.86711,243957.356623,NO STANDING 10AM-6PM MON THRU FRI,0,NaN,NaN,NaN
1,NaN,NaN,"<Point><coordinates>-73.9223354763,40.83625571...",1,11494928,B,P-132428,4,SP-672G,45,NaN,NaN,1005740.86711,243957.356623,EXCEPT (RIDER),0,NaN,NaN,NaN
2,NaN,NaN,"<Point><coordinates>-73.9223354763,40.83625571...",1,11494929,B,P-132428,5,SP-579G,45,NaN,NaN,1005740.86711,243957.356623,AMBULETTE,0,NaN,NaN,NaN
3,NaN,NaN,"<Point><coordinates>-73.922330083,40.836352065...",1,11494930,B,P-132428,6,SP-287BA,80,NaN,S,1005742.32839,243992.461212,NO STANDING 10AM-6PM MON THRU FRI (SINGLE ARROW),0,NaN,0,NaN
4,NaN,NaN,"<Point><coordinates>-73.922330083,40.836352065...",1,11494931,B,P-132428,7,SP-672G,80,NaN,NaN,1005742.32839,243992.461212,EXCEPT (RIDER),0,NaN,NaN,NaN
